# Points of Interest Scoring
This notebook handles the loading of datasets and runs a scoring algorithm on a grid array

In [1]:
import pandas as pd
import ast
import math
import numpy as np
from pathlib import Path
from get_bounds import *
from misc_utility import *
from plotting_utility import *

### Dataset Retrieval and Preprocessing

In [2]:
def getRestaurants(city):
    df = pd.read_csv('../res/datasets/csv/' + city + '/restaurant.csv', sep = ',', header=None, names=['lat', 'long', 'name'])
    df['type'] = 'restaurant'
    return df

In [3]:
def getCommunityCentres(city):
    df = pd.read_csv('../res/datasets/csv/' + city + '/community_centre.csv', sep = ',', header=None, names=['lat', 'long', 'name'])
    df['type'] = 'community_centre'
    return df

In [4]:
def getFoodCourts(city):
    df = pd.read_csv('../res/datasets/csv/' + city + '/food_court.csv', sep = ',', header=None, names=['lat', 'long', 'name'])
    df['type'] = 'food_court'
    return df

In [5]:
def getLibraries(city):
    df = pd.read_csv('../res/datasets/csv/' + city + '/library.csv', sep = ',', header=None, names=['lat', 'long', 'name'])
    df['type'] = 'library'
    return df

In [6]:
def getUniversities(city):
    df = pd.read_csv('../res/datasets/csv/' + city + '/university.csv', sep = ',', header=None, names=['lat', 'long', 'name'])
    df['type'] = 'university'
    return df

### Scoring Parameters

In [7]:
STD_DEV = 0.6
PT_DENSITY = 100
SCORE_MIN_THRESHOLD = 0.01

### Scoring Functions

In [8]:
def score(latStation, longStation, latOrigin, longOrigin, stdDev):

    stationCor = (latStation, longStation)

    totalScore = 0

    for Lat, Long in zip(latOrigin, longOrigin):
        origin = (Lat, Long)
        r = getDistance(origin, stationCor)
        # this score function simply uses the traditional gaussian distribution formula,
        # except it extracts all the common parts to be multiplied last for efficiency.
        score =  math.exp(-0.5*((r**2) / (stdDev**2)))
        totalScore += score

    #adjustFactor = 210 * 0.9974
    #totalScore = (totalScorePre/adjustFactor)*100

    # Multiply common part in gaussian distribution formula
    totalScore *= (1/(stdDev * math.sqrt(2 * math.pi)))

    return totalScore

In [9]:
def generateGridScores(poi, std_dev, upperLatBound, bottomLatBound, leftLongBound, rightLongBound):
    gridPoints = pd.DataFrame(columns = ['lat', 'long', 'score'])
    latStep = round((upperLatBound-bottomLatBound)/(PT_DENSITY-1), 4)
    longStep = round((rightLongBound-leftLongBound)/(PT_DENSITY-1), 4)
    quarterPoint = bottomLatBound+latStep*PT_DENSITY/4
    midPoint = bottomLatBound+latStep*PT_DENSITY/2
    tfPoint = bottomLatBound+latStep*3*PT_DENSITY/4

    for lat in np.linspace(bottomLatBound, upperLatBound, num=PT_DENSITY):
        if (abs(lat - quarterPoint) <= 0.005):
            print("25% complete")
        if (abs(lat - midPoint) <= 0.005):
            print("50% complete")
        if (abs(lat - tfPoint) <= 0.005):
            print("75% complete")
        for long in np.linspace(leftLongBound, rightLongBound, num=PT_DENSITY):
            pointScore = score(lat, long, poi['lat'], poi['long'], std_dev);

            pointScore = pointScore if pointScore > SCORE_MIN_THRESHOLD else 0.0;

            pointScoreDict = {'lat' : [lat], 'long' : [long], 'score' : [pointScore]};
            pointScoreDf = pd.DataFrame(data=pointScoreDict)
            gridPoints = pd.concat([gridPoints, pointScoreDf], ignore_index = True)

    return gridPoints

### Score

In [10]:
def scoreGridPoints(city, std_dev):
    restaurants = getRestaurants(city)
    community_centres = getCommunityCentres(city)
    food_courts = getFoodCourts(city)
    libraries = getLibraries(city)
    universities = getUniversities(city)

    pointsOfInterest = pd.concat([restaurants, community_centres, food_courts, libraries, universities])
    [upperLatBound, bottomLatBound, leftLongBound, rightLongBound] = getBounds(pointsOfInterest['lat'], pointsOfInterest['long'], 4)
    return generateGridScores(pointsOfInterest, std_dev, upperLatBound, bottomLatBound, leftLongBound, rightLongBound)

In [11]:
#YYZ_GridPoints = scoreGridPoints("toronto", STD_DEV)
#YVR_GridPoints = scoreGridPoints("vancouver", STD_DEV)
#YYC_GridPoints = scoreGridPoints("calgary")

### Export

In [12]:
def exportScoresToCsv(gridPoints, name):
    filepath = Path('../res/grid_points/' + name + '.csv')
    filepath.parent.mkdir(parents=True, exist_ok=True)
    gridPoints.to_csv(filepath)

In [13]:
def exportDataFrameToCoordsTxt(dataFrame, name):
    with open(name + '_coords.txt', 'w') as f:
        for lat in dataFrame['lat']:
            f.write(str(lat))
            f.write(',')
    f.write("\n")
    for long in dataFrame['long']:
        f.write(str(long))
        f.write(',')

In [14]:
#exportScoresToCsv(YYZ_GridPoints, 'yyz_grid_points')
#exportScoresToCsv(YVR_GridPoints, 'yvr_grid_points')
#exportScoresToCsv(YYC_GridPoints, 'yyc_grid_points')

In [15]:
def overnightOperation():
    STD_DEVS = [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.96, 0.97]
    for sd in STD_DEVS:
        YYZ_GridPoints = scoreGridPoints("toronto", sd)
        exportScoresToCsv(YYZ_GridPoints, 'yyz_grid_points_' + str(sd))

        YVR_GridPoints = scoreGridPoints("vancouver", sd)
        exportScoresToCsv(YVR_GridPoints, 'yvr_grid_points_' + str(sd))

        YYC_GridPoints = scoreGridPoints("calgary", sd)
        exportScoresToCsv(YYC_GridPoints, 'yyc_grid_points_' + str(sd))

In [16]:
#plotGridPoints3D(YVR_GridPoints)

In [ ]:
overnightOperation()

25% complete
25% complete
25% complete
25% complete
50% complete
50% complete
50% complete
50% complete
75% complete
75% complete
75% complete
75% complete
25% complete
25% complete
25% complete
25% complete
25% complete
25% complete
25% complete
25% complete
25% complete
50% complete
50% complete
50% complete
50% complete
50% complete
50% complete
50% complete
50% complete
50% complete
75% complete
75% complete
75% complete
75% complete
75% complete
75% complete
75% complete
75% complete
75% complete
25% complete
25% complete
25% complete
50% complete
50% complete
50% complete
75% complete
75% complete
75% complete
25% complete
25% complete
25% complete
25% complete
50% complete
50% complete
50% complete
50% complete
75% complete
75% complete
75% complete
75% complete
25% complete
25% complete
25% complete
25% complete
25% complete
25% complete
25% complete
25% complete
25% complete
50% complete
50% complete
50% complete
50% complete
50% complete
50% complete
50% complete
50% complete